In [25]:
import sys 
import os
import pandas as pd
from scipy import stats
import numpy as np

In [26]:
sys.path.append(os.path.abspath('../scripts'))

In [27]:
from task33 import *
from load_data import *

In [28]:
query = "SELECT * FROM xdr_data;"

df = load_data_from_postgres(query)
print(df.head(10))


Connected to the database


/home/abrham/Documents/kifiya/week2/Telecom-Data-Analysis/scripts/load_data.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


      Bearer Id            Start  Start ms              End  End ms  \
0  1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1  1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2  1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3  1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4  1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   
5  1.311448e+19  4/12/2019 21:37     439.0   4/25/2019 8:08   553.0   
6  1.311448e+19   4/13/2019 8:41     612.0   4/25/2019 8:16   168.0   
7  1.304243e+19   4/14/2019 2:11     592.0   4/25/2019 2:26   512.0   
8  1.311448e+19  4/14/2019 12:48     121.0  4/25/2019 10:22   960.0   
9  1.304243e+19   4/15/2019 0:32       0.0   4/25/2019 0:40   284.0   

   Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0  1823652.0  2.082014e+14   3.366496e+10  3.552121e+13   
1  1365104.0  2.082019e+14   3.368185e+10  3.579401e+13   
2  1361762.0  2.082003e+14   3.376063e+10

In [31]:
#Cleaning the columns with high missing values
df_clean = df.drop(['Nb of sec with 37500B < Vol UL',
                    'Nb of sec with 6250B < Vol UL < 37500B','Nb of sec with 1250B < Vol UL < 6250B',
                    'Nb of sec with 125000B < Vol DL',
                    'Nb of sec with 31250B < Vol DL < 125000B',
                    'Nb of sec with 6250B < Vol DL < 31250B',
                   'TCP DL Retrans. Vol (Bytes)',
                    'TCP UL Retrans. Vol (Bytes)',
                    'HTTP DL (Bytes)',
                    'HTTP UL (Bytes)'], axis=1)
df_clean=df_clean.dropna(subset=['End'])
df_clean.shape
print (f"The shape of the dataframe after cleaning some data is {df_clean.shape}")



The shape of the dataframe after cleaning some data is (150000, 45)


In [32]:
df_clean.isna().sum()

Bearer Id                            0
Start                                0
Start ms                             0
End                                  0
End ms                               0
Dur. (ms)                            0
IMSI                                 0
MSISDN/Number                        0
IMEI                                 0
Last Location Name                1152
Avg RTT DL (ms)                      0
Avg RTT UL (ms)                      0
Avg Bearer TP DL (kbps)              0
Avg Bearer TP UL (kbps)              0
DL TP < 50 Kbps (%)                  0
50 Kbps < DL TP < 250 Kbps (%)       0
250 Kbps < DL TP < 1 Mbps (%)        0
DL TP > 1 Mbps (%)                   0
UL TP < 10 Kbps (%)                  0
10 Kbps < UL TP < 50 Kbps (%)        0
50 Kbps < UL TP < 300 Kbps (%)       0
UL TP > 300 Kbps (%)                 0
Activity Duration DL (ms)            0
Activity Duration UL (ms)            0
Dur. (ms).1                          0
Handset Manufacturer     

In [33]:
df_clean.sample(5)

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
58583,1.304243e+19,4/26/2019 0:22,613.0,4/27/2019 1:19,821.0,89782.0,2.082014e+14,3.366027e+10,8.654080e+13,T88349B,...,21046751.0,20226339.0,6630669.0,15435889.0,492666890.0,2929915.0,252887231.0,10369694.0,51540599.0,526207720.0
140220,1.304243e+19,4/29/2019 3:24,322.0,4/30/2019 3:24,317.0,86399.0,2.082018e+14,3.366743e+10,8.637690e+13,D88350C,...,20978992.0,10988201.0,7424248.0,15274210.0,285010326.0,12355580.0,483325218.0,4939867.0,46958540.0,324211581.0
66105,7.349883e+18,4/26/2019 7:33,495.0,4/27/2019 9:59,567.0,95118.0,2.082014e+14,3.366105e+10,3.557671e+13,D76121C,...,14789779.0,15727535.0,7593088.0,484859.0,456993364.0,11835349.0,731777265.0,7587901.0,38624059.0,491910512.0
58468,1.311448e+19,4/25/2019 23:56,1.0,4/27/2019 10:15,163.0,123540.0,2.082014e+14,3.366896e+10,3.593041e+13,L70273C,...,18560954.0,17226805.0,5419874.0,8397713.0,734183436.0,16315193.0,607714022.0,12580671.0,56717926.0,762079099.0
109713,1.311448e+19,4/27/2019 17:46,125.0,4/29/2019 8:14,373.0,138485.0,2.082010e+14,3.366509e+10,3.557301e+13,D21521B,...,16566318.0,296896.0,11853052.0,3140271.0,707501862.0,12984954.0,592204087.0,13930966.0,33313338.0,745966604.0


In [34]:
df_clean['Start'] = pd.to_datetime(df['Start'])
df_clean['End'] = pd.to_datetime(df['End'])
df_clean.isna().sum()

Bearer Id                            0
Start                                0
Start ms                             0
End                                  0
End ms                               0
Dur. (ms)                            0
IMSI                                 0
MSISDN/Number                        0
IMEI                                 0
Last Location Name                1152
Avg RTT DL (ms)                      0
Avg RTT UL (ms)                      0
Avg Bearer TP DL (kbps)              0
Avg Bearer TP UL (kbps)              0
DL TP < 50 Kbps (%)                  0
50 Kbps < DL TP < 250 Kbps (%)       0
250 Kbps < DL TP < 1 Mbps (%)        0
DL TP > 1 Mbps (%)                   0
UL TP < 10 Kbps (%)                  0
10 Kbps < UL TP < 50 Kbps (%)        0
50 Kbps < UL TP < 300 Kbps (%)       0
UL TP > 300 Kbps (%)                 0
Activity Duration DL (ms)            0
Activity Duration UL (ms)            0
Dur. (ms).1                          0
Handset Manufacturer     

In [35]:
df_clean.sample(5)

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
79594,7.277826e+18,2019-04-27 07:23:00,365.0,2019-04-28 00:19:00,967.0,60984.0,2.082014e+14,3.366215e+10,3.566111e+13,D84756A,...,11471670.0,16524433.0,17137566.0,581020.0,250188676.0,2003998.0,161403082.0,15077980.0,35463344.0,290410920.0
96317,1.304243e+19,2019-04-27 16:36:00,434.0,2019-04-28 00:04:00,93.0,26905.0,2.082014e+14,3.364191e+10,8.662640e+13,L42014A,...,10263233.0,17316333.0,20050372.0,5103712.0,475777884.0,5902015.0,204107189.0,9892621.0,38841916.0,513851658.0
56800,1.304243e+19,2019-04-25 18:47:00,939.0,2019-04-27 17:36:00,20.0,168532.0,2.082017e+14,3.369819e+10,3.588751e+13,D93808B,...,5205863.0,9201799.0,4366410.0,19705282.0,186481386.0,10493971.0,390394574.0,10359208.0,52378687.0,202761273.0
96804,1.311448e+19,2019-04-27 16:43:00,35.0,2019-04-28 00:05:00,380.0,26509.0,2.082014e+14,3.366851e+10,8.639550e+13,T79693C,...,17088002.0,21554534.0,16560407.0,21591316.0,404025586.0,13688883.0,814045951.0,325808.0,60646594.0,453985065.0
106922,1.304243e+19,2019-04-27 12:34:00,777.0,2019-04-29 10:15:00,120.0,164449.0,2.082019e+14,3.376273e+10,3.575751e+13,T30774C,...,21089304.0,14978257.0,13779245.0,15060087.0,126727299.0,9294160.0,410841575.0,6288914.0,47268934.0,169727779.0


In [36]:
df_clean.head()

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311448e+19,2019-04-04 12:01:00,770.0,2019-04-25 14:35:00,662.0,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,...,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,1.311448e+19,2019-04-09 13:04:00,235.0,2019-04-25 08:15:00,606.0,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,...,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,1.311448e+19,2019-04-09 17:42:00,1.0,2019-04-25 11:58:00,652.0,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,...,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,1.311448e+19,2019-04-10 00:31:00,486.0,2019-04-25 07:36:00,171.0,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,...,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,1.311448e+19,2019-04-12 20:10:00,565.0,2019-04-25 10:40:00,954.0,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,...,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


In [37]:
# Identify columns with non-numeric values
non_numeric_columns = df_clean.select_dtypes(exclude=['number']).columns

# Drop columns with non-numeric values or handle them appropriately
df_numeric = df_clean.drop(columns=non_numeric_columns)

# Calculate skewness for numeric columns
skewness = df_numeric.skew(axis=0)


In [38]:
missing_columns = ["Avg RTT DL (ms)",
                   "Avg RTT UL (ms)",
                   "DL TP < 50 Kbps (%)",
                   "50 Kbps < DL TP < 250 Kbps (%)",
                   "250 Kbps < DL TP < 1 Mbps (%)",
                   "DL TP > 1 Mbps (%)",
                   "UL TP < 10 Kbps (%)",
                   "10 Kbps < UL TP < 50 Kbps (%)",
                   "50 Kbps < UL TP < 300 Kbps (%)",
                   "UL TP > 300 Kbps (%)",
                   "Nb of sec with Vol DL < 6250B",
                   "Nb of sec with Vol UL < 1250B"]

In [39]:
def fix_missing_mean(df, col):
    df[col] = df[col].fillna(df[col].mean())
    return df[col]

def fix_missing_median(df, col):
    df[col] = df[col].fillna(df[col].median())
    return df[col]

def fix_missing_mode(df, col):
    df[col] = df[col].fillna(df_clean[col].mode()[0])
    return df[col]

In [40]:
for i in missing_columns:
    df_clean[i] = fix_missing_median(df_clean, i)
df_clean["Last Location Name"] = fix_missing_mode(df_clean,"Last Location Name")

In [41]:
df_clean.isna().sum()

Bearer Id                           0
Start                               0
Start ms                            0
End                                 0
End ms                              0
Dur. (ms)                           0
IMSI                                0
MSISDN/Number                       0
IMEI                                0
Last Location Name                  0
Avg RTT DL (ms)                     0
Avg RTT UL (ms)                     0
Avg Bearer TP DL (kbps)             0
Avg Bearer TP UL (kbps)             0
DL TP < 50 Kbps (%)                 0
50 Kbps < DL TP < 250 Kbps (%)      0
250 Kbps < DL TP < 1 Mbps (%)       0
DL TP > 1 Mbps (%)                  0
UL TP < 10 Kbps (%)                 0
10 Kbps < UL TP < 50 Kbps (%)       0
50 Kbps < UL TP < 300 Kbps (%)      0
UL TP > 300 Kbps (%)                0
Activity Duration DL (ms)           0
Activity Duration UL (ms)           0
Dur. (ms).1                         0
Handset Manufacturer              571
Handset Type

In [47]:
df_4 = df[['IMSI',
           'Avg RTT DL (ms)',
            'Avg RTT UL (ms)',
            'TCP DL Retrans. Vol (Bytes)',
            'TCP UL Retrans. Vol (Bytes)',
            'Handset Type',
            'Avg Bearer TP DL (kbps)',
            'Avg Bearer TP UL (kbps)']]

In [48]:
df_4.head()

,IMSI,Avg RTT DL (ms),Avg RTT UL (ms),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Handset Type,Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps)
0,2.082014e+14,42.000000,5.000000,2.080991e+07,759658.664811,Samsung Galaxy A5 Sm-A520F,23.0,44.0
1,2.082019e+14,65.000000,5.000000,2.080991e+07,759658.664811,Samsung Galaxy J5 (Sm-J530),16.0,26.0
2,2.082003e+14,109.795706,17.662883,2.080991e+07,759658.664811,Samsung Galaxy A8 (2018),6.0,9.0
3,2.082014e+14,109.795706,17.662883,2.080991e+07,759658.664811,undefined,44.0,44.0
4,2.082014e+14,109.795706,17.662883,2.080991e+07,759658.664811,Samsung Sm-G390F,6.0,9.0


In [49]:
df_4=df_4.dropna(subset=['IMSI'])

In [50]:
df_4.isna().sum()

IMSI                             0
Avg RTT DL (ms)                  0
Avg RTT UL (ms)                  0
TCP DL Retrans. Vol (Bytes)      0
TCP UL Retrans. Vol (Bytes)      0
Handset Type                   572
Avg Bearer TP DL (kbps)          0
Avg Bearer TP UL (kbps)          0
dtype: int64

In [51]:
numeric_cols = df_4.select_dtypes(include=[np.number]).columns
df_4[numeric_cols].skew(axis=0, skipna=True)

IMSI                            41.124164
Avg RTT DL (ms)                 69.705161
Avg RTT UL (ms)                 31.530109
TCP DL Retrans. Vol (Bytes)     24.840677
TCP UL Retrans. Vol (Bytes)    141.035684
Avg Bearer TP DL (kbps)          2.589445
Avg Bearer TP UL (kbps)          4.503428
dtype: float64

In [52]:
col_clean = [['Avg RTT DL (ms)',
 'Avg RTT UL (ms)',
 'TCP DL Retrans. Vol (Bytes)',
 'TCP UL Retrans. Vol (Bytes)',
 'Avg Bearer TP DL (kbps)',
 'Avg Bearer TP UL (kbps)']]

In [53]:
#Replace missing values with mean or median(thumb rule)
for i in col_clean:
    df_4[i] = fix_missing_median(df_4, i)
df_4["Handest Type"] = fix_missing_mode(df_4, "Handset Type")

In [54]:
df_4.head()

,IMSI,Avg RTT DL (ms),Avg RTT UL (ms),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Handset Type,Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),Handest Type
0,2.082014e+14,42.000000,5.000000,2.080991e+07,759658.664811,Samsung Galaxy A5 Sm-A520F,23.0,44.0,Samsung Galaxy A5 Sm-A520F
1,2.082019e+14,65.000000,5.000000,2.080991e+07,759658.664811,Samsung Galaxy J5 (Sm-J530),16.0,26.0,Samsung Galaxy J5 (Sm-J530)
2,2.082003e+14,109.795706,17.662883,2.080991e+07,759658.664811,Samsung Galaxy A8 (2018),6.0,9.0,Samsung Galaxy A8 (2018)
3,2.082014e+14,109.795706,17.662883,2.080991e+07,759658.664811,undefined,44.0,44.0,undefined
4,2.082014e+14,109.795706,17.662883,2.080991e+07,759658.664811,Samsung Sm-G390F,6.0,9.0,Samsung Sm-G390F


In [56]:
#Aggregate per customer
Groupby = df_4.groupby(['IMSI'])

In [57]:
Avg_RTT = Groupby['Avg RTT DL (ms)'].sum() + Groupby['Avg RTT UL (ms)'].sum()
Avg_TCP = Groupby['TCP DL Retrans. Vol (Bytes)'].sum()+Groupby['TCP UL Retrans. Vol (Bytes)'].sum()

Avg_TP = Groupby['Avg Bearer TP DL (kbps)'].sum()+Groupby['Avg Bearer TP UL (kbps)'].sum()
Handset = Groupby['Handset Type'].first()

In [59]:
new2 = pd.concat([Avg_RTT,
                  Avg_TCP,
                  Avg_TP,
                  Handset
                ], axis="columns", sort=True)

new2 = new2.rename(columns={0: 'Avg RTT'})
new2 = new2.rename(columns={1: 'TCP'})
new2 = new2.rename(columns={2: 'Avg TP'})
new2 = new2.rename(columns={3: 'Handset type'})
new2.index.name = 'Customer ID'
new2.reset_index(inplace=True) 
#Top 10 TCP values
new2.sort_values(by='TCP', ascending=False)[['Customer ID','TCP']].head(10)

,Customer ID,TCP
57608,2.082016e+14,1.108931e+10
83403,2.082018e+14,8.354209e+09
11273,2.082009e+14,7.941860e+09
81533,2.082018e+14,7.374899e+09
82687,2.082018e+14,6.043978e+09
11278,2.082009e+14,5.879989e+09
81445,2.082018e+14,5.593366e+09
27808,2.082014e+14,5.566929e+09
11181,2.082009e+14,5.195785e+09
83180,2.082018e+14,5.179459e+09


In [60]:
# Top 10 most frequent TCP values
new2['TCP'].value_counts().head(10)

TCP
2.156957e+07    47199
4.313915e+07     7475
6.470872e+07     1033
2.081121e+07      364
8.627829e+07      252
2.081124e+07      168
7.609887e+05      146
4.238078e+07       82
2.081123e+07       81
7.623187e+05       80
Name: count, dtype: int64

In [61]:
# Top 10 bottom TCP values
new2.sort_values(by='TCP', ascending=False)[['TCP']].tail(10)

,TCP
102611,182.0
14282,179.0
52330,177.0
12644,176.0
5962,176.0
60467,143.0
56004,134.0
107029,129.0
6407,128.0
29703,97.0


In [62]:
# Top 10 RTT values
new2.sort_values(by='Avg RTT', ascending=False)[['Avg RTT']].head(10)

,Avg RTT
57608,275894.030982
60037,96924.000000
59325,64670.000000
97640,54848.000000
66139,46021.000000
82981,37084.000000
82657,36304.000000
40882,27278.000000
26711,26300.000000
7172,25715.000000


In [63]:
# Top 10 most frequent RTT values
new2['Avg RTT'].value_counts().head(10)

Avg RTT
127.458589    16370
29.000000      2738
39.000000      2168
254.917178     1909
38.000000      1447
30.000000      1419
40.000000      1390
28.000000      1329
49.000000      1202
31.000000      1054
Name: count, dtype: int64

In [64]:
# Top 10 bottom RTT values
new2.sort_values(by='Avg RTT', ascending=False)[['Avg RTT']].tail(10)

,Avg RTT
12935,10.0
48343,9.0
1368,9.0
90083,8.0
70977,6.0
13477,5.0
36675,4.0
73881,2.0
4705,0.0
75244,0.0


In [65]:
# Top 10  TP values
new2.sort_values(by='Avg TP', ascending=False)[['Avg TP']].head(10)

,Avg TP
57608,7.120197e+06
51374,9.022220e+05
97724,8.404240e+05
84141,7.983420e+05
66279,6.908150e+05
69738,6.768580e+05
61987,6.187970e+05
43375,5.641790e+05
15187,5.591970e+05
76374,5.503520e+05


In [66]:
# Top 10 bottom TP datasets
new2.sort_values(by='Avg TP', ascending=False)[['Avg TP']].tail(10)

,Avg TP
95608,0.0
56757,0.0
39532,0.0
25143,0.0
29681,0.0
102007,0.0
39040,0.0
51909,0.0
40074,0.0
10774,0.0


In [67]:
# Top 10 bottom TP datasets
new2.sort_values(by='Avg TP', ascending=False)[['Avg TP']].tail(10)

,Avg TP
95608,0.0
56757,0.0
39532,0.0
25143,0.0
29681,0.0
102007,0.0
39040,0.0
51909,0.0
40074,0.0
10774,0.0


In [69]:
new2.to_pickle("data.pkl") 